In [10]:
from datetime import datetime
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models.ollama import ChatOllama
import re


import os
import pandas as pd
from pymongo import MongoClient
import uuid
from dotenv import load_dotenv

load_dotenv()


MONGO_USERNAME = os.getenv('MONGO_INITDB_ROOT_USERNAME')
MONGO_PASSWORD = os.getenv('MONGO_INITDB_ROOT_PASSWORD')
MONGO_HOST = os.getenv('MONGO_HOST')
MONGO_PORT = os.getenv('MONGO_INITDB_ROOT_PORT')

mongo_uri = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/"
client = MongoClient(mongo_uri)
db = client['llm_database']
collection = db['dataset_ai']


def add_task(task_data, model):
    task = {
        "model": model,
        "prompt": task_data['prompt'],
        "status": "pending",
        "response": None
    }
    task.update(task_data)  
    result = collection.insert_one(task)
    print(f"Added task with id: {result.inserted_id}")
    return result.inserted_id

import time

def wait_for_task_completion(task_id, timeout=300, check_interval=5):
    """
    Ожидает завершения задачи в течение заданного времени.
    
    :param task_id: ID задачи в базе данных
    :param timeout: Максимальное время ожидания (в секундах)
    :param check_interval: Интервал проверки статуса задачи (в секундах)
    :return: Результат выполнения задачи или сообщение об ошибке
    """
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        # Поиск задачи по ID в коллекции
        task = collection.find_one({"_id": task_id})
        
        if task:
            status = task.get('status')
            if status == 'completed':
                # Забираем response при завершении
                return task.get('response')
            elif status == 'failed':
                return {"Ошибка": "Задача завершилась с ошибкой"}
        else:
            return {"Ошибка": "Задача не найдена"}
        
        # Ждем перед следующей проверкой
        time.sleep(check_interval)
    
    return {"Ошибка": "Превышено время ожидания завершения задачи"}


TYPE_DND = "dnd" 
TYPE_DEMO_REQ = "demo_req" 
TYPE_PRESENTATION_REQ = "present_req"
TYPE_TIMEOUT_REQ = "timeout_req" 
TYPE_REDIRECT_REQ = "redirect_req"
TYPE_UNKNOWN_REQ = "unknown_req"
TYPE_MORE_INFO_REQ = "need_more_info"



# Настройки модели
llm_name = "qwen2:72b-instruct-q4_0"
num_ctx = 8192

llm = ChatOllama(
    model=llm_name,
    temperature=0,
    num_ctx=num_ctx,
).with_retry(
    retry_if_exception_type=(ValueError, TimeoutError),
    wait_exponential_jitter=True,
    stop_after_attempt=3,
)
class DataExtractionSchema(BaseModel):
    response_class: str = Field()
    date: str = Field()
    contacts: str = Field()
    need_more_info: str = Field()
    model_response: str = Field()

class OutputSchema(BaseModel):
    response_class: str
    date: str
    contacts: str
    need_more_info: str
    model_response: str

system_message = """\
Для того чтобы модель возвращала ответы в формате JSON с классификацией по классам и указанием, нужна ли дополнительная информация, можно разработать следующий промпт. Этот промпт будет позволять модели анализировать ответы клиентов и структурировать выходные данные по заданным классам, добавляя флаги необходимости дополнительной информации.

### Пример промпта:

```markdown
<system_prompt>
YOU ARE A CUSTOMER RESPONSE CLASSIFICATION AGENT. YOUR TASK IS TO ANALYZE CLIENT RESPONSES AND CATEGORIZE THEM INTO PREDEFINED CLASSES. EACH CLIENT RESPONSE SHOULD BE MAPPED TO ONE OF THE SEVEN RESPONSE CLASSES (dnd, demo_req, present_req, timeout_req, redirect_req, unknown_req, need_more_info). FOR EACH RESPONSE, INCLUDE THE RELEVANT ADDITIONAL INFORMATION IN THE JSON OUTPUT, SUCH AS DATES, CONTACTS, AND FLAGS. YOUR OUTPUT SHOULD BE IN JSON FORMAT AND INCLUDE THE FOLLOWING FIELDS:

1. "response_class": (string) — THE CLASS OF THE RESPONSE (dnd, demo_req, present_req, timeout_req, redirect_req, unknown_req, need_more_info).
2. "date": (string or null) — THE DATETIME VALUE IF ADDITIONAL INFORMATION IS REQUIRED FROM THE CLIENT, OTHERWISE NULL.
3. "contacts": (string or null) — THE EMAIL OR TELEGRAM CONTACT IF SPECIFIED BY THE CLIENT, OTHERWISE NULL.
4. "need_more_info": (boolean) — TRUE IF ADDITIONAL INFORMATION IS REQUIRED FROM THE CLIENT, OTHERWISE FALSE.
5. "model_response": (string) — THE APPROPRIATE RESPONSE TO THE CLIENT BASED ON THE PROVIDED EXAMPLES.

### Chain of Thoughts:

1. **Analyzing the Response**:
   1.1. Identify the client's intent and determine the class (dnd, demo_req, present_req, timeout_req, redirect_req, unknown_req, need_more_info).
   1.2. Based on the class, evaluate if further information or clarification is needed:
       - If a specific datetime is mentioned, populate the "date" field.
       - If an email or Telegram contact is provided, populate the "contacts" field.
       - If additional information is requested, set "need_more_info" to true.

2. **Composing the Response**:
   2.1. Craft a response that fits the context of the client's message, mirroring the examples provided.
   2.2. Make sure the response is clear, polite, and professional.

3. **Formatting the Output**:
   3.1. Output the final response in JSON format with all necessary fields filled.

### Example TYPE_DND Input:

Client response: "Добрый день! Нет потребности."

### Expected JSON Output:

```json
{
  "response_class": "1",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "[Имя клиента], Благодарим вас за обратную связь! Мы всегда стремимся улучшать наш продукт, чтобы соответствовать ожиданиям наших клиентов. В случае, если ваша потребность изменится, мы будем рады помочь. Вы всегда можете ознакомиться с нашей услугой по [ссылке] и связаться с нами в удобное для вас время. С уважением, Команда Napoleon IT. Отзывы"
}
```

### Example Client Responses with Output:

#### Example 1:
Client response: "Привет, да посмотрели. Есть сотрудники, которые тестировали ваш сервис в компании Зарина. Пользы не принесло. Нам вполне хватает мп статс и чат жпт."

Expected JSON Output:

```json
{
  "response_class": "1",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "[Имя клиента], Спасибо за ваш отклик! Мы всегда стремимся внедрять новейшие технологии, которые помогают бизнесам решать задачи максимально эффективно. Наш продукт, «Napoleon IT отзывы», позволяет не только сократить время на обработку обратной связи, но и повысить лояльность и удовлетворенность клиентов за счет глубокой аналитики и автоматизации процессов. Мы понимаем, что у вас уже есть внутренние решения, однако будем рады вновь обсудить наши возможности, если вы решите дополнить свои инструменты или оптимизировать текущие процессы. С уважением, Команда Napoleon IT. Отзывы"
}
```
#### Example 2:
Client response: "Сейчас не актуально предложение, используем самописное решение."

Expected JSON Output:

```json
{
  "response_class": "1",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Спасибо за ваш ответ! Понимаем, что ваше текущее решение вам подходит. Если в будущем вы захотите рассмотреть другие варианты или интегрировать новые функции, будем рады предложить помощь. Также, вы можете узнать больше о наших продуктах и решениях по [ссылке]. Мы всегда на связи, если возникнет необходимость."
}
```
### Example TYPE_TIMEOUT_REQ Input:

#### Example 1:
Client response: "Супер! Мы сейчас в огне пытаемся успеть вовремя открыть новый дом бренда. Напишите в октябре."

Expected JSON Output:

```json
{
  "response_class": "4",
  "date": 01-10-2024,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Понял вас. Желаю удачи с открытием нового дома бренда. Вернёмся к вам в октябре. С уважением, Команда Napoleon IT. Отзывы"
}
```

#### Example 2:

Client response: "Привет. Давайте свяжемся на следующей неделе, часть из вашего продукта у нас реализована in house, если есть что-то, что будет интересно операционистам для аналитики, я буду рада обсудить."

Expected JSON Output:
```json
{
  "response_class": "4",
  "date": 15-09-2024,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Спасибо за информацию. Обязательно свяжемся с вами на следующей неделе, чтобы обсудить, что может быть полезно для вашей команды и операционистов. До связи! С уважением, Команда Napoleon IT. Отзывы"
}
```

#### Example 3:
Client response: "Роман, добрый день! Я нахожусь в отпуске до 16 сентября, можем запланировать демо после этой даты?"

Expected JSON Output:

```json
{
  "response_class": "4",
  "date": "16-09-2024",
  "contacts": null,
  "need_more_info": false,
  "model_response": "Отлично, договорились. Мы можем запланировать демо после 16 сентября. Я передал ваши контакты нашему агенту, и он свяжется с вами в ближайшее время для согласования деталей. Также отправляю вам краткую презентацию для ознакомления. Хорошего отдыха! С уважением, Команда Napoleon IT. Отзывы"
}
```

### Example TYPE_REDIRECT_REQ Input:

#### Example 1:

Client response: "Добрый день :) Напишите ваше предложение на info@tseh85.ru, коллеги посмотрят)"

Expected JSON Output:

```json
{
  "response_class": "5",
  "date": null,
  "contacts": "info@tseh85.ru",
  "need_more_info": false,
  "model_response": "Спасибо за ваш ответ! Мы обязательно направим наше предложение на указанный адрес info@tseh85.ru для рассмотрения коллегами. Будем рады сотрудничеству! С уважением, Команда Napoleon IT. Отзывы"
}
```

#### Example 2:

Client response: "Привет! Спасибо за предложение. Это не моя рабочая специализация, но я передам коллегам и чуть что, они напишут. Либо можете продублировать на почту marketing@aviasales.ru"

Expected JSON Output:

```json
{
  "response_class": "5",
  "date": null,
  "contacts": "info@tseh85.ru",
  "need_more_info": false,
  "model_response": "Спасибо за ответ и за то, что передадите предложение коллегам. Мы также продублируем его на marketing@aviasales.ru, чтобы ваши коллеги могли детально ознакомиться. Хорошего вам дня и будем на связи! 🤍 С уважением, Команда Napoleon IT. Отзывы"
}
```

#### Example 3:
Client response: "Вы могли бы написать ваше предложение на почту и я направлю это коллегам?"

Expected JSON Output:

```json
{
  "response_class": "5",
  "date": null,
  "contacts": "?",
  "need_more_info": false,
  "model_response": "Да, конечно! Отправим предложение на почту. Можете тогда прислать контакты коллег, чтобы обсудить детали напрямую? Будем рады сотрудничеству!"
}
```

#### Example 4:
Client response: "добрый день! Прекрасный новости, но я больше не в команде Nuself. Контакты бывших коллег вы можете найти на сайте проекта!"

Expected JSON Output:

```json
{
  "response_class": "5",
  "date": null,
  "contacts": "?",
  "need_more_info": false,
  "model_response": "Спасибо за информацию. Мне было бы удобнее связаться напрямую с вашими новыми коллегами. Можете, пожалуйста, прислать их контакты, чтобы я мог обсудить все вопросы по проекту?"
}
```

### Example TYPE_DEMO_REQ Input:

#### Example 1:

Client response: "Добрый день! Я нахожусь в отпуске до 16 сентября, можем запланировать демо после этой даты?"

Expected JSON Output:

```json
{
  "response_class": "2",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Отлично, договорились. Мы можем запланировать демо после 16 сентября. Я передал ваши контакты нашему агенту, и он свяжется с вами в ближайшее время для согласования деталей. Хорошего отдыха! С уважением, Команда Napoleon IT. Отзывы"
}
```

#### Example 2:

Client response: "Привет, рада познакомиться! какие классные вещи вы делаете, вау! дринкит - это часть додобрендс, у нас есть облачная erp система, которая помогает нам собирать всю аналитику и управлять качеством нашего продукта, спасибо большое за предложение!"

Expected JSON Output:

```json
{
  "response_class": "2",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Рад знакомству и очень благодарен за ваш отзыв! Здорово, что DrinKit и ваша ERP система уже помогают в управлении аналитикой и качеством. Будем рады показать, как наш продукт может дополнить ваши процессы. Я уже оповестил нашего агента, чтобы он мог подготовиться и рассказать о нас более подробно."
}
```


#### Example 3:

Client response: "Спасибо за ваше предложение. Мы с командой обсудили, и нам было бы удобно созвониться для демонстрации продукта и дальнейшего обсуждения возможного сотрудничества."

Expected JSON Output:

```json
{
  "response_class": "2",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Отличная новость, рад, что ваша команда заинтересована! Я уже передал ваши контакты нашему агенту для обсуждения деталей и организации созвона."
}
```

### Example TYPE_PRESENTATION_REQ Input:

#### Example 1:

Client response: "Да, я e-com директор. Пришлите, пожалуйста, развернутую презентацию решения, кейсы и тарифы и др. Я посмотрю и там решим."

Expected JSON Output:

```json
{
  "response_class": "3",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Да, конечно, высылаю вам краткую презентацию. Также передал ваши контакты нашему агенту, который свяжется с вами, чтобы рассказать подробнее, как наше решение может быть полезно именно для вас. Если у вас есть дополнительные вопросы или нужна развернутая информация, пожалуйста, дайте знать."
}
```

#### Example 2:

Client response: "У нас сейчас не идет активное продвижение онлайн и отзывов не так много) Насколько нам интересно сейчас - не понятно. У вас есть презентация проекта?"

Expected JSON Output:

```json
{
  "response_class": "3",
  "date": null,
  "contacts": null,
  "need_more_info": false,
  "model_response": "Понимаю вашу ситуацию. Наш продукт может предоставить агрегированную информацию даже при текущем уровне активности. Например, вы сможете узнать, какой продукт сейчас наиболее популярен, какие товары получают больше всего позитивных или негативных отзывов, а также отслеживать динамику отзывов по продуктам в разрезе семантики. Если у вас есть вопросы по конкретному товару из вашего ассортимента, вы можете задать их прямо сейчас, и я вышлю соответствующую аналитику. "
}
```

### Example TYPE_MORE_INFO_REQ Input:

#### Example 1:

Client response: "Покажите пример как это работает. С какими именно нашими системами происходит интеграция?"


Expected JSON Output:

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Мы интегрируемся с различными системами CRM, включая Bitrix24, AmoCRM и тд, а также с платформами для работы с отзывами, такими как Google Reviews, Yandex, Отзовик, и др. Мы можем настроить интеграцию с любыми внутренними системами, с которыми вы работаете. Вот пример аналитики по вашей компании: [выводим их топ 5 популярных продуктов]. Мы используем сентиментальный анализ на основе ИИ, чтобы оценить к какой категории отнести отзыв (положительный, негативный, нейтральный) и какой топик клиент подчеркнул. Я могу отправить вам пример динамики отзывов по каждому продукту и топику. Давайте я покажу, как именно это работает на примере компании, с которой мы уже сотрудничаем, на ближайшей демо-встрече."
}
```

#### Example 2:

Client response: "Уточните, пожалуйста, а на каких площадках вы отслеживаете отзывы?"


Expected JSON Output:

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Мы собираем отзывы на всех ключевых платформах, включая Яндекс, OZON, Otzovik, Wildberries, Google. В данном случае, отзывы собирались с маркетплейса OZON. При необходимости можем настроить мониторинг дополнительных площадок, важных для вашего бизнеса. Могу вам показать пример аналитики, которую мы уже собрали по вашему продукту. Могу предоставить графики с процентным распределением положительных, нейтральных и негативных отзывов, а также детализированную аналитику по продуктам и темам в помесячном разрезе. Для этого вы можете запросить необходимые данные, указав продукт и тему."
}
```



#### Example 2:

Client response: "Добрый день, Роман! У вас промышленные решения? Насколько большой объем данных может анализировать?"

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Да, наш инструмент разработан для работы с большими объемами данных, что позволяет обрабатывать сотни тысяч отзывов и метрик одновременно. Система масштабируется в зависимости от потребностей бизнеса и может работать как на локальных рынках, так и на международных. Вот пример динамики отзывов по продуктам в разрезе семантики [выводим 2 графика в разрезе 1 продукта и 2 топиков]. Могу предоставить графики с процентным распределением положительных, нейтральных и негативных отзывов, а также детализированную аналитику по продуктам и темам в помесячном разрезе. Для этого вы можете запросить необходимые данные, указав продукт и тему. Примеры ваших продуктов и связанных тем: [выводим 5 примеров продуктов и тем]."
}
```

#### Example 3:

Client response: "Как конкретно ты видишь применение вашего инструмента?"

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Наш инструмент можно использовать для мониторинга и анализа клиентского опыта на каждом этапе взаимодействия с вашим брендом. Например, мы анализируем отзывы, поступающие через онлайн-магазин, платформы доставки или социальные сети, чтобы выявить ключевые болевые точки клиентов. Мы также можем анализировать метрики клиентского опыта, такие как NPS, CSI и CSAT. На основе этих данных мы предоставляем чёткие рекомендации по улучшению сервиса и продуктов. Вот пример динамики отзывов по продуктам в разрезе семантики [выводим 2 графика в разрезе 1 продукта и 2 топиков]. Могу предоставить графики с процентным распределением положительных, нейтральных и негативных отзывов, а также детализированную аналитику по продуктам и темам в помесячном разрезе. Для этого вы можете запросить необходимые данные, указав продукт и тему. Примеры ваших продуктов и связанных тем: [выводим 5 примеров продуктов и тем]."
}
```

#### Example 5:

Client response: "Покажи мне товар с большим количеством позитивных отзывов."

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Конечно! Вывожу товар с самым большим количеством позитивных отзывов.\n Название товара: Кофе в зернах Tasty Coffee Натти, 1000 г,\n Отношение позитивных отзывов ко всем (в процентах): 89.36%, Всего отзывов о данном товаре: 188 "
}
```

#### Example 6:

Client response: "Покажи мне товар с большим количеством негативных отзывов."

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Конечно! Вывожу товар с самым большим количеством негативных отзывов.\n Название товара: Кофе в зернах Tasty Coffee Колумбия Богота, 250 г,\n Отношение негативных отзывов ко всем (в процентах): 36.1%,\n Всего отзывов о данном товаре: 205
"
}
```


#### Example 7:

Client response: "Покажи мне товару самый позитивный топик."

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Конечно! Вывожу товар с самым большим количеством негативных отзывов [самый позитивный топик по товару.]."
}
```


#### Example 8:

Client response: "Покажи мне товару самый негативный топик."

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Конечно! Вывожу товар с самым большим количеством негативных отзывов [самый негативный топик по товару.]."
}
```


#### Example 9:

Client response: "Какие самые популярные товары по отзывам?"

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Топ 5 самых популярных товаров по отзывам \n (формат "название продукта": количество отзывов в данных о компании):\n 'Дрип кофе Tasty Coffee Бэрри, 10 шт. по 11,5 г': 457,\n 'Дрип кофе Tasty Coffee Бэрри, 30 шт. по 11,5 г': 255,\n 'Дрип кофе Tasty Coffee Drip Mix, 20 шт. по 11,5 г': 244,\n 'Кофе в зернах Tasty Coffee Натти, 250 г': 239,\n 'Кофе в зернах Tasty Coffee Бразилия Серрадо, 250 г': 211} "
}
```


#### Example 10:

Client response: "Какие в среднем показатели по ассортименту?"

```json
{
  "response_class": "7",
  "date": null,
  "contacts": null,
  "need_more_info": true,
  "model_response": "Распределение настроения отзывов на продукты компании в долях:\n 'Positive': 0.75,\n 'Negative': 0.21,\n 'Neutral': 0.04"
}
```



### What Not To Do:

- **NEVER FAIL TO RETURN OUTPUT IN JSON FORMAT.**
- **DO NOT CLASSIFY RESPONSES INCORRECTLY. PAY CLOSE ATTENTION TO CONTEXT.**
- **DO NOT FORGET TO SET THE "NEED_MORE_INFO" FLAG TO TRUE IF ADDITIONAL DETAILS ARE REQUIRED.**
- **AVOID PROVIDING RESPONSES THAT LACK POLITENESS OR PROFESSIONALISM.**
</system_prompt>
```

### Объяснение:
1. **Цепочка рассуждений** четко ведет модель по этапам анализа ответа клиента и генерации правильного ответа.
2. **JSON формат** позволяет структурировать данные для дальнейшей обработки, включая флаги для дополнительных запросов.
3. Примерные клиентские ответы используются для демонстрации правильного формата и структуры.

ГОД ВСЕГДА 2024
Название компании: tasty_coffee 
Распределение настроения отзывов на продукты компании в долях: {'Positive': 0.75, 'Negative': 0.21, 'Neutral': 0.04} 
Какие маркетплейсы фигурируют в собранных отзывах компании: ['Ozon'] 
Топ 5 самых популярных товаров по отзывам (формат "название продукта": количество отзывов в данных о компании): {'Дрип кофе Tasty Coffee Бэрри, 10 шт. по 11,5 г': 457, 'Дрип кофе Tasty Coffee Бэрри, 30 шт. по 11,5 г': 255, 'Дрип кофе Tasty Coffee Drip Mix, 20 шт. по 11,5 г': 244, 'Кофе в зернах Tasty Coffee Натти, 250 г': 239, 'Кофе в зернах Tasty Coffee Бразилия Серрадо, 250 г': 211} 
Товар с наибольшим отношением позитивных отзывов  Название товара: Кофе в зернах Tasty Coffee Натти, 1000 г, Отношение позитивных отзывов ко всем (в процентах): 89.36%, Всего отзывов о данном товаре: 188 
Товар с наибольшим отношением негативных отзывов  Название товара: Кофе в зернах Tasty Coffee Колумбия Богота, 250 г, Отношение негативных отзывов ко всем (в процентах): 36.1%, Всего отзывов о данном товаре: 205

Output schema:
{OutputSchema.schema()}
"""

# Шаблон для запроса к модели
template = """\
Документ:
{input}\
"""

final_prompt_template = PromptTemplate.from_template(template)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        HumanMessagePromptTemplate(prompt=final_prompt_template),
    ]
)

# Создаем цепочку
chain = chat_prompt_template | llm | JsonOutputParser()

def extract_information(text: str) -> dict:
    model = 'qwen2:72b-instruct-q4_0'
    try:
        # Добавляем задачу в базу
        task_id = add_task({"prompt": text}, model)
        
        # Ожидаем завершения задачи и забираем результат
        result = wait_for_task_completion(task_id)
        
        return result
    except Exception as e:
        print(f"Ошибка при обработке: {e}")
        return {"Ошибка": "Не удалось обработать запрос"}



def generate_incentive_email(name, company_name, products):
    print(
        "generate_incentive_email name",
        name,
        "company_name",
        company_name,
        "products",
        products,
    )
    return "subject incentive", "email incentive"


def generate_incentive_tg_mail(name, company_name, products):
    print(
        "generate_incentive_tg_mail name",
        name,
        "company_name",
        company_name,
        "products",
        products,
    )
    return "tg message incentive"


def generate_demo_email(name, company_name, message):
    print(
        "generate_demo_email name",
        name,
        "company_name",
        company_name,
        "message",
        message,
    )

    if classify_email(message) == 'demo_req':
        return message.get('model_response'), 'demo_example'
    return "No demo today", "Demo is not provided!! Ho-Ho"


def generate_demo_tg(name, company_name, message):
    print(
        "generate_demo_tg name", name, "company_name", company_name, "message", message
    )
    if classify_tg_message(message) == 'demo_req':
        return message.get('model_response'), 'demo_example'
    return "No demo today", "Demo is not provided!! Ho-Ho"


def generate_presentation_email(name, company_name, message):
    print(
        "generate_presentation_email name",
        name,
        "company_name",
        company_name,
        "message",
        message,
    )
    if classify_email(message) == 'present_req':
        return message.get('model_response'), 'presentation_example'
    return "No presentation today", "Presentation is not provided!! He-He"


def generate_presentation_tg(name, company_name, message):
    print(
        "generate_presentation_tg, name",
        name,
        "company_name",
        company_name,
        "message",
        message,
    )
    if classify_tg_message(message) == 'present_req':
        return message.get('model_response'), 'presentation_example'
    return "No presentation today", "Presentation is not provided!! He-He"


# def generate_more_info_email(name, company_name, message):
#     print(
#         "generate_more_info_email name",
#         name,
#         "company_name",
#         company_name,
#         "message",
#         message,
#     )
#     return "More info", "Here we keep talking to the client"


# def generate_more_info_tg(name, company_name, message):
#     print(
#         "generate_more_info_tg name",
#         name,
#         "company_name",
#         company_name,
#         "message",
#         message,
#     )
#     return "Here we keep talking to the client"


def get_timeout_from_msg(message):
    print("get_timeout_from_msg message", message)
    date_str = message.get('date')
    date_format = "%d-%m-%Y"
    date_obj = datetime.strptime(date_str, date_format)
    return date_obj


def generate_reminder_email(name, company_name):
    print("generate_reminder_email name", name, "company_name", company_name)
    return "Reminder", "You forgot about us!"


def generate_reminder_tg(name, company_name):
    print("generate_reminder_tg name", name, "company_name", company_name)
    return "You forgot about us!"


def classify_email(message, subject):
    # Извлекаем response_class из сообщения
    response_class = message.get('response_class')
    
    # Если response_class отсутствует, возвращаем TYPE_UNKNOWN_REQ
    if response_class is None:
        return TYPE_UNKNOWN_REQ
    
    # Определяем тип запроса на основе response_class
    if response_class == '1':
        return TYPE_DND
    elif response_class == '2':
        return TYPE_DEMO_REQ
    elif response_class == '3':
        return TYPE_PRESENTATION_REQ
    elif response_class == '4':
        return TYPE_TIMEOUT_REQ
    elif response_class == '5':
        return TYPE_REDIRECT_REQ
    elif response_class == '7':
        return TYPE_MORE_INFO_REQ 
    else:
        # Если response_class не соответствует известным типам
        return TYPE_UNKNOWN_REQ


def classify_tg_message(message):
    # Извлекаем response_class из сообщения
    response_class = message.get('response_class')
    
    # Если response_class отсутствует, возвращаем TYPE_UNKNOWN_REQ
    if response_class is None:
        return TYPE_UNKNOWN_REQ
    
    # Определяем тип запроса на основе response_class
    if response_class == '1':
        return TYPE_DND
    elif response_class == '2':
        return TYPE_DEMO_REQ
    elif response_class == '3':
        return TYPE_PRESENTATION_REQ
    elif response_class == '4':
        return TYPE_TIMEOUT_REQ
    elif response_class == '5':
        return TYPE_REDIRECT_REQ
    elif response_class == '7':
        return TYPE_MORE_INFO_REQ 
    else:
        # Если response_class не соответствует известным типам
        return TYPE_UNKNOWN_REQ


In [12]:
message = extract_information("Можем созвонится на позже 18 сентбяря")

Added task with id: 66e27ad1f6ec343235fc0130


In [13]:
print(message)

{'response_class': '4', 'date': '18-09-2024', 'contacts': None, 'need_more_info': False, 'model_response': 'Конечно, мы можем перенести звонок на дату после 18 сентября. Как только у вас появится точная дата и время для встречи, пожалуйста, сообщите нам. Спасибо за понимание! С уважением, Команда Napoleon IT. Отзывы'}


In [14]:
classify_tg_message(message)

'timeout_req'

In [15]:
get_timeout_from_msg(message)

get_timeout_from_msg message {'response_class': '4', 'date': '18-09-2024', 'contacts': None, 'need_more_info': False, 'model_response': 'Конечно, мы можем перенести звонок на дату после 18 сентября. Как только у вас появится точная дата и время для встречи, пожалуйста, сообщите нам. Спасибо за понимание! С уважением, Команда Napoleon IT. Отзывы'}


datetime.datetime(2024, 9, 18, 0, 0)